# 1. AWS Overview

In this lesson, we are going to explore the basics of cloud computing. We will be using Amazon Web Services (AWS) as the cloud provider. A cloud provider is a service that provides compute resources to you. In this lesson, we will explore the following:

  - What is AWS?
  - What is cloud computing?
  - What Services are available in AWS?
  - How to create a RDS instance

First of all, we need to go to the AWS console and create a new account. To do this, go to the [AWS Console](https://console.aws.amazon.com/console/home), and create a new account. You will need a credit card to pay for the AWS services, but worry not! You can use a free service tier.

Once created, go to the [AWS Console](https://console.aws.amazon.com/console/home) and select the Services tab. You will see a list of services that are available

![](images/AWS_Services.png)

There are plenty of services in AWS, you can even manage your satellite server in the cloud! We only have bugdet for providing a few satellites, so instead, in this introduction we will focus on :

- [Amazon RDS](https://aws.amazon.com/rds/) - Relational Database Service
- [Amazon S3](https://aws.amazon.com/s3/) - Simple Storage Service
- [Amazon EC2](https://aws.amazon.com/ec2/) - Elastic Compute Cloud

For both RDS and EC2, we will need to establish a virtual network. This will limit the range of IP addresses that can be used to access the service. You can simply create a Security Group in AWS, and assign a range of IP addresses to it. To do so, go to the search bar and type VPC. Then click on the Security Groups tab.

<p float="left">
  <img src="images/VPC_search.png" width="350" />
  <img src="images/VPC_Security.png" width="350" /> 
</p>

Click on the `Create security group` button. And in the following window, enter a name for the security group and a description. After that, add inbound rules to the security group. You can add inbound rules to allow specific IP addresses. In this notebook, we will connect to the RDS instance from our local machine. So, we will allow the local machine to connect to the RDS ports.

<p float="left">
  <img src="images/Security_group_2.png" width="350" />
  <img src="images/Security_group_3.png" width="350" /> 
</p>

In `Source`, choose `My IP`, so you are the only one with access to the RDS instance.

> Important: If you have a dynamic IP, this will work the first time, but next time your connection will be refused. In that case, change the source to Anywhere IPv4.

As mentioned, this Security Group is useful for the RDS instance. We will also create <ins>another</ins> Security Group <ins>__for the EC2 instances__</ins>, where the Inbound rules are:

- HTTPS: With source equal to Anywhere IPv4
- HTTP: With source equal to Anywhere IPv4
- SSH: With source equal to My IP




Now that the security groups are created, we can create our RDS instance. 

# AWS RDS

AWS RDS is a service that allows you to create a database in the cloud. It is a highly scalable database that can be used for a variety of purposes. It allows creating databases of different types, such as PostgreSQL, MySQL, Oracle, and Amazon Aurora. Since we already know how it works, let's create a PostgreSQL database.

Go to the [AWS Console](https://console.aws.amazon.com/console/home) and select the Services tab. Then click on the RDS tab. After that, click on `'Create database'`

![](images/Create_RDS.png)

In the next window, select PostgreSQL as the type of database and select Version 12.5-R1. If you want to go with, you will need to pay!

![](images/Create_RDS2.png)

Then give your instance a memorable name:

![](images/Create_RDS3.png)

In the DB instance class, choose db.t2.micro, which is free! In the connectivity options, select publicly accessible. and in Existing Security Group, select the Security Group that we created earlier.

![](images/Create_RDS4.png)

Click on Create, and wait for it to be created. It might take a while. Once finished, you will see the status has changed to `available`. Now, click on the instance ID, and you will see the details of the instance. You have to take note of the Endpoint, which is the IP address of the database.

![](images/Create_RDS5.png)

And now you are ready to connect to the database! The default user and database are `postgres` and `postgres`. 

In [19]:
from sqlalchemy import create_engine
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'aicoredb.c8k7he1p0ynz.us-east-1.rds.amazonaws.com' # Change it for your AWS endpoint
USER = 'postgres'
PASSWORD = 'Cosamona94'
PORT = 5432
DATABASE = 'postgres'
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")



Run the following cell to see if everything works:

In [20]:
engine.connect()

Let's create a new table in our database. We are going to put the famous iris dataset into our database. The iris dataset is a set of measurements of flowers from 150 species. It is a famous dataset that is used in many machine learning applications.

In [1]:
!pip install sklearn

In [21]:
from sklearn.datasets import load_iris
import pandas as pd
data = load_iris()
iris = pd.DataFrame(data['data'], columns=data['feature_names'])
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [22]:
iris.to_sql('iris_dataset', engine, if_exists='replace')

Unfortunately, AWS RDS doesn't allow see the tables you created, but to can still access to them using pgAdmin or SQLAlchemy itself. 

In [23]:
df = pd.read_sql_table('iris_dataset', engine)

In [24]:
df.head()

,index,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,5.1,3.5,1.4,0.2
1,1,4.9,3.0,1.4,0.2
2,2,4.7,3.2,1.3,0.2
3,3,4.6,3.1,1.5,0.2
4,4,5.0,3.6,1.4,0.2


And if you go to pgAdmin, you will see the table that you created. First, establish the connection to the database:

![](images/pgAdmin4.png)

And then observe that, in the postgres database, iris_dataset is there.

![](images/pgAdmin4_2.png)

# Summary

- You learned the basics of cloud computing.
- You saw how to create Security groups that can limit the range of IP addresses that can access the service.
- You saw how to create a RDS instance.
- You saw how to connect to the RDS instance using SQLAlchemy.
- You saw how to create a table in the database.
- Finally, you saw how to connect to the database using pgAdmin.